In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df_amz=pd.read_excel('/content/drive/MyDrive/PROJECT/datasets/amazon_cells_labelled_.xlsx')

In [ ]:
df_amz

,Sl. Nos,Text,Sentiment
0,1,So there is no way for me to plug it in here i...,0
1,2,"Good case, Excellent value.",1
2,3,Great for the jawbone.,1
3,4,Tied to charger for conversations lasting more...,0
4,5,The mic is great.,1
...,...,...,...
995,996,The screen does get smudged easily because it ...,0
996,997,What a piece of junk.. I lose more calls on th...,0
997,998,Item Does Not Match Picture.,0
998,999,The only thing that disappoint me is the infra...,0


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
df_amz['Text'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000 entries, 0 to 999
Series name: Text
Non-Null Count  Dtype 
--------------  ----- 
1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [ ]:
df_amz.isnull().sum()

Sl. Nos      0
Text         0
Sentiment    0
dtype: int64

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    print(tokens)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
df_amz['processed_text'] = df_amz['Text'].apply(preprocess_text)

In [ ]:
df_amz['processed_text'] = df_amz['Text'].str.replace('[^\w\s]', '').str.lower()

<ipython-input-11-8c18ffdd7315>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_amz['processed_text'] = df_amz['Text'].str.replace('[^\w\s]', '').str.lower()


In [ ]:
df_amz.head()

,Sl. Nos,Text,Sentiment,processed_text
0,1,So there is no way for me to plug it in here i...,0,so there is no way for me to plug it in here i...
1,2,"Good case, Excellent value.",1,good case excellent value
2,3,Great for the jawbone.,1,great for the jawbone
3,4,Tied to charger for conversations lasting more...,0,tied to charger for conversations lasting more...
4,5,The mic is great.,1,the mic is great


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(df_amz['processed_text'])
y = df_amz['Sentiment']

In [ ]:
X[0,]

<1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
y

0      0
1      1
2      1
3      0
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: Sentiment, Length: 1000, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.88      0.82        93
           1       0.88      0.77      0.82       107

    accuracy                           0.82       200
   macro avg       0.82      0.82      0.82       200
weighted avg       0.83      0.82      0.82       200



In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)


In [ ]:
cm

array([[82, 11],
       [25, 82]])